In [31]:
import csv
from twitterscraper import query_tweets
import re

demoFile = 'demo.csv'
#Write headers to file
with open(demoFile,'w') as csvFile:
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow(['Timestamp', 'Text', 'User', 'Likes', 'Retweets'])
    
#Save the retrieved tweets to file:
for tweet in query_tweets("$EURUSD OR $USDJPY OR $GBPUSD OR $USDCAD OR $USDCHF OR $AUDUSD ", 50):
    tweetText = tweet.text
    tweetText = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\w+:\/\/\S+)"," ",tweetText).split())
    tweetTime = tweet.timestamp
    retweets = tweet.retweets
    likes = tweet.likes
    userName = tweet.user
    with open(demoFile, 'a', encoding='utf-8') as csvFile:
        csvWriter = csv.writer(csvFile)
        csvWriter.writerow([tweetTime,tweetText, userName, likes, retweets]) 
print("Finished collecting data")

Finished collecting data


In [29]:
import pandas as pd
from nltk.corpus import stopwords

df = pd.read_csv(demoFile, encoding='utf-8')

#preprocess data
def preprocess():
    stopwordList = set(stopwords.words('english'))
    #remove 'not' from list of stopwords for sentiment analysis
    stopwordList.remove("not")
    
    df['PP Text'] = df['Text'].map(lambda x: ' '.join(re.sub("(RT)|(@[A-Za-z0-9]+)|(pic.twitter.com\/+\S+)|([^0-9A-Za-z.!? \t]+)|(\w+:\/\/\S+)"," ",x).split()))
    #Remove duplicated words
    df['PP Text'] = df['PP Text'].apply(lambda x: " ".join(sorted(set(x.split()), key=x.split().index)))

    df['PP Text'] = df['PP Text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stopwordList)]) )
    df.to_csv(demoFile, encoding='utf-8', index=False)
    
preprocess()
print("Finished preprocessing data")


Finished preprocessing data


In [30]:
pd.options.display.max_colwidth = -1
df[['Text', 'PP Text']]
#df.loc[20]

,Text,PP Text
0,$GBPUSD 1.3355 exit pic.twitter.com/pcXcL17ToQ,GBPUSD 1.3355 exit
1,Today's $AUDUSD #Forex signal- pic.twitter.com/MhPvIzPzk0,Today AUDUSD Forex signal
2,$GBPUSD - Líneas de resistencia y soporte GBP/USD - …,GBPUSD L neas de resistencia soporte GBP USD
3,（備忘録）ユーロドルに戻ります。 －－－ ★２１日：３０分足・１時間足複合上昇波動逆張り【円口座】 $EURUSD(selllimit1.18700-1.18000/lot12.00) pic.twitter.com/4UTYGaa6H3,EURUSD selllimit1.18700 1.18000 lot12.00
4,USDJPY pair 1H speculation 20-11-2017 long** - $USDJPY chart …,USDJPY pair 1H speculation 20 11 2017 long chart
5,USDJPY pair 1H speculation 20-1\u2026 $USDJPY …,USDJPY pair 1H speculation 20 1 u2026
6,wait breakout candle M15 $USDJPY $FXY,wait breakout candle M15 USDJPY FXY
7,$USDJPY so close to freedom pic.twitter.com/Ima9cvkNL6,USDJPY close freedom
8,Euro Dollaro Consolida 20-11-2017 $EURUSD …,Euro Dollaro Consolida 20 11 2017 EURUSD
9,Eur/usd short on top $EURUSD …,Eur usd short top EURUSD


In [14]:
import nltk
from nltk.stem import *
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

row = "USDJPY market alert 4 week low 112.46"

lemmatizer = WordNetLemmatizer()
sid = SentimentIntensityAnalyzer()

#Convert between the PennTreebank tags to simple Wordnet tags
def convertWN(tag):

    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None
 
sentiment = 0.0
tagged=nltk.pos_tag(nltk.word_tokenize(row))
print("Tweet: ", row)
print(tagged)
for word, tag in tagged:
    wnTag = convertWN(tag)
    if wnTag not in (wn.NOUN, wn.ADJ, wn.ADV):
        continue
 
    lemma = lemmatizer.lemmatize(word, pos=wnTag)

    if not lemma:
        continue
 
    synsets = wn.synsets(lemma, pos=wnTag)
    if not synsets:
        continue
    
    # Take the first sense, the most common
    synset = synsets[0]
    swnscore = swn.senti_synset(synset.name())
    sentiment += (swnscore.pos_score() - swnscore.neg_score())
    print("Lemma: ", lemma, end="\t" ) 

    print("Sentiment : ",  swnscore.pos_score() - swnscore.neg_score())
print()
print("Total SentiWordNet Sentiment: ", sentiment)

### Calculate VADER Sentiment
ss = sid.polarity_scores(row)    
print("Vader Sentiment: ", ss['compound'])


Tweet:  USDJPY market alert 4 week low 112.46
[('USDJPY', 'JJ'), ('market', 'NN'), ('alert', 'VBD'), ('4', 'CD'), ('week', 'NN'), ('low', 'JJ'), ('112.46', 'CD')]
Lemma:  market	Sentiment :  0.0
Lemma:  week	Sentiment :  0.0
Lemma:  low	Sentiment :  -0.25

Total SentiWordNet Sentiment:  -0.25
Vader Sentiment:  0.0258


In [72]:
### Perform SentiWordNet sentiment analysis on Twitter Data from the last week of October
import pandas as pd
from pandas.tseries.offsets import BDay
import nltk
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn

filename = '/Users/mitalipalsikar/Documents/year3project/Daily Twitter/twitterData.csv'
df = pd.read_csv(filename, encoding='utf-8')

#take values of last week of October
df = df[(df['Timestamp'] > '2017-10-13 00:00:00') & (df['Timestamp'] < '2017-10-27 23:00:00')]

isBusinessDay = BDay().onOffset
match_series = pd.to_datetime(df['Timestamp']).map(isBusinessDay)
df = df[match_series]
svalues = []
lvalues=[]
lemmatizer = WordNetLemmatizer()

#Convert between the PennTreebank tags to simple Wordnet tags
def convertWN(tag):

    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

#Break tweet into synsets and get score of each sentence
def swnSentiment(sentence):
    sentiment = 0.0
    tagged=nltk.pos_tag(nltk.word_tokenize(sentence))
    for word, tag in tagged:
        wnTag = convertWN(tag)
        if wnTag not in (wn.NOUN, wn.ADJ, wn.ADV):
            continue
 
        lemma = lemmatizer.lemmatize(word, pos=wnTag)
        if not lemma:
            continue

        synsets = wn.synsets(lemma, pos=wnTag)
        if not synsets:
            continue
        # Take the first sense, the most common
        synset = synsets[0]
        swnscore = swn.senti_synset(synset.name())
        sentiment += (swnscore.pos_score() - swnscore.neg_score())

    return sentiment

#iterate through each row of df and get tweet
def sentiWordNet():
    i = 0;
    fsentiment = 0.0
    for row in df.values:
        sentence = row[5]
        sd = {}
        date = row[0]
        fsentiment = swnSentiment(sentence);
        sd['Tweet'] = sentence
        sd['Date'] = date
        sd['Sentiment'] = fsentiment
        svalues.append(sd)
    sdf = pd.DataFrame(svalues)
    print("Finished sentiment analysis for Dataset")
    return sdf

def vaderSentiment():

    sid = SentimentIntensityAnalyzer()
    i = 0;
    for row in df.values:
        sentence = row[5]
        date = row[0]
        ss = sid.polarity_scores(sentence)    
        ss['Tweet'] = sentence
        ss['Date'] = date
        lvalues.append(ss)

    vdf = pd.DataFrame(lvalues)
    print("Finished sentiment analysis for Dataset")
    return vdf
#sdf = sentiWordNet()
sdf = vaderSentiment()

Finished sentiment analysis for Dataset


In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
fxFile = '/Users/mitalipalsikar/Documents/year3project/Daily Rates/fxhistoricaldataGBPUSD.csv'

#Get sentiment analysis for all tweets talking about $GBPUSD
gbpusd = sdf[sdf['Tweet'].str.contains("GBPUSD") ]
gbpusd.set_index('Date', inplace=True)
gbpusd.index = pd.to_datetime(gbpusd.index)

fx = pd.read_csv(fxFile)
isBusinessDay = BDay().onOffset
match_series = pd.to_datetime(fx['Time']).map(isBusinessDay)
fx = fx[match_series]
#FX values for GBPUSD for same date range
fx = fx[(fx['Time'] >= '2017-10-13') & (fx['Time'] < '2017-10-28 00:00:00')]
fx.set_index('Time', inplace=True)
fx.index = pd.to_datetime(fx.index)

def plotChanges( fxval, senval):
    
    senval= senval.resample('D').std()
    fxval= fxval.resample('D').std()
    
    #senval = senval.replace([np.inf, -np.inf], np.nan)
    #fxval = fxval.replace([np.inf, -np.inf], np.nan)

    fxval = fxval.fillna(method='ffill')
    senval = senval.fillna(method='ffill')

    fxfig = fxval.plot( y = 'Close', marker='o', color='r', use_index=True)
    swnfig = senval.plot( y = 'compound', marker='o', use_index=True)
    fxfig.get_figure().savefig('/Users/mitalipalsikar/Documents/year3project/presentation1/demoGBPUSD.png')
    swnfig.get_figure().savefig('/Users/mitalipalsikar/Documents/year3project/presentation1/demoswn.png')

plotChanges(fx, gbpusd)